# Parking with Hindsight Experience Replay

##  Warming up
We start with a few useful installs and imports:

In [ ]:
# Install environment and agent
!pip install git+https://github.com/alexandrashaw10/adv_robotics_parking_env.git
!pip install git+https://github.com/DLR-RM/stable-baselines3
!pip install sb3-contrib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/alexandrashaw10/adv_robotics_parking_env.git to /tmp/pip-req-build-xyjt1_7e
  Running command git clone --filter=blob:none --quiet https://github.com/alexandrashaw10/adv_robotics_parking_env.git /tmp/pip-req-build-xyjt1_7e
  Resolved https://github.com/alexandrashaw10/adv_robotics_parking_env.git to commit 325d3f5d180432dbe492527bc7e7d528064096e4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 97.6 MB/s eta 0:00:00
  Created wheel for highway-env: filename=highway_env-1.5-py3-none-any.whl size=103521 sha256=0f67296da17c25630dc70056f3716b6f323f0ed1e9e8c966294678c03ab5cc72
  Stored in directory: /tmp/pip-ephem-wheel-cache-mr0ng22h/wheels/26/d1/dd/a49d961ab585da36facd08640e470c

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=222ec38ab9d38d4506eda34f10e6ee35eb36a1be7b00d5e3755f4b6e4e840a81
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools


In [ ]:
# Environment
import gym
import highway_env

# Agent
from stable_baselines3 import HerReplayBuffer, SAC
from sb3_contrib import TQC

In [ ]:
# Import wandb
import wandb
wandb.login()


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
config = {
      "env_name": "parking-parkedVehicles-v0",
      "learning_rate": 1e-3,
      "algorithm": "TQC",
      "HER": True,
      "batch_size": 1024,
      "gamma": 0.95,
      "epochs": 10,
      "online_sampling": True,
      "goal_selection_strategy": "future",
      "n_sampled_goal": 4,
      "total_timesteps": 5e2,
    }

In [ ]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="advanced-robo-parking-env",
    
    # track hyperparameters and run metadata
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
)

## Training

In [ ]:
# !apt-get update && apt-get install python-opengl xvfb
# !pip install pyvirtualdisplay stable_baselines3[extra] wandb
# from pyvirtualdisplay import Display
# virtual_display = Display(visible=0, size=(1400, 900))
# virtual_display.start()
!pip install gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg

from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from gym.wrappers import RecordVideo
from pathlib import Path
import base64
from tqdm.notebook import trange

display = Display(visible=0, size=(1400, 900))
display.start()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached PyVirtualDisplay-3.0-py3-none-any.whl (15 kB)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-2build1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
xvfb is already the newest version (2:1.20.13-1ubuntu1~20.04.6).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [ ]:
from wandb.integration.sb3 import WandbCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder

def make_env():
  env = gym.make('parking-parkedVehicles-v0')
  env = Monitor(env)

  return env

env = DummyVecEnv([make_env])
env = VecVideoRecorder(env, f"videos/baseline", record_video_trigger=lambda x: x % 2000 == 0, video_length=200)

her_kwargs = dict(n_sampled_goal=4, goal_selection_strategy='future', online_sampling=True, max_episode_length=10)
# You can replace TQC with SAC agent
model = TQC('MultiInputPolicy', env, replay_buffer_class=HerReplayBuffer,
            replay_buffer_kwargs=her_kwargs, verbose=1, buffer_size=int(1e6),
            learning_rate=1e-3,
            gamma=0.95, batch_size=1024, tau=0.05,
            policy_kwargs=dict(net_arch=[512, 512, 512]),
            tensorboard_log=f"runs/baseline")

model.learn(total_timesteps=config["total_timesteps"], callback=WandbCallback())
wandb.finish()

Using cuda device
Logging to runs/baseline/TQC_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32       |
|    ep_rew_mean     | -265     |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 6        |
|    time_elapsed    | 20       |
|    total_timesteps | 128      |
| train/             |          |
|    actor_loss      | -2.77    |
|    critic_loss     | 1.27     |
|    ent_coef        | 0.974    |
|    ent_coef_loss   | -0.0876  |
|    learning_rate   | 0.001    |
|    n_updates       | 27       |
---------------------------------
Saving video to /content/videos/baseline/rl-video-step-0-to-step-200.mp4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 33       |
|    ep_rew_mean     | -141     |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 8        |
|    fps             | 7    

global_step,▁▅█
rollout/ep_len_mean,▁█▃
rollout/ep_rew_mean,▁█▃
rollout/success_rate,▁▁▁
time/fps,▁▃█
train/actor_loss,█▃▁
train/critic_loss,█▁▄
train/ent_coef,█▄▁
train/ent_coef_loss,█▄▁
train/learning_rate,▁▁▁
global_step,388


## Visualize a few episodes

We first define a simple helper function for visualization of episodes:

In [ ]:
!pip install gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg

from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from gym.wrappers import RecordVideo
from pathlib import Path
import base64
from tqdm.notebook import trange

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
    html = []
    for mp4 in Path("video").glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay 
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  freeglut3 libpython2-stdlib python2 python2-minimal
Suggested packages:
  python-tk python-numpy libgle3 python2-doc
The following NEW packages will be installed:
  freeglut3 libpython2-stdlib python-opengl python2 python2-minimal xvfb
0 upgraded, 6 newly installed, 0 to remove and 21 not upgraded.
Need to get 1,400 kB of archives.
After this operation, 8,330 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 python2-minimal amd64 2.7.17-2ubuntu4 [27.5 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal


Test the policy

In [ ]:
env = gym.make('parking-parkedVehicles-v0')
env = RecordVideo(env, video_folder='./videos', episode_trigger=lambda e: True)
env.unwrapped.set_record_video_wrapper(env)
for episode in trange(3, desc="Test episodes"):
    obs, done = env.reset(), False
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
env.close()
show_video()

Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]